<a href="https://colab.research.google.com/github/chiyanglin-AStar/python_stock_study/blob/main/%E5%8F%B0%E8%82%A1%E6%AF%8F%E6%97%A5%E8%B3%87%E6%96%99.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
import numpy as np
import requests
import pandas as pd
import datetime

#   http://www.twse.com.tw/exchangeReport/STOCK_DAY?date=20180817&stockNo=2330  取一個月的股價與成交量
def get_stock_history(date, stock_no):
    quotes = []
    url = 'http://www.twse.com.tw/exchangeReport/STOCK_DAY?date=%s&stockNo=%s' % ( date, stock_no)
    r = requests.get(url)
    data = r.json()
    return transform(data['data'])  #進行資料格式轉換

def transform_date(date):
        y, m, d = date.split('/')
        return str(int(y)+1911) + '/' + m  + '/' + d  #民國轉西元
    
def transform_data(data):
    data[0] = datetime.datetime.strptime(transform_date(data[0]), '%Y/%m/%d')
    data[1] = int(data[1].replace(',', ''))  #把千進位的逗點去除
    data[2] = int(data[2].replace(',', ''))
    data[3] = float(data[3].replace(',', ''))
    data[4] = float(data[4].replace(',', ''))
    data[5] = float(data[5].replace(',', ''))
    data[6] = float(data[6].replace(',', ''))
    data[7] = float(0.0 if data[7].replace(',', '') == 'X0.00' else data[7].replace(',', ''))  # +/-/X表示漲/跌/不比價
    data[8] = int(data[8].replace(',', ''))
    return data

def transform(data):
    return [transform_data(d) for d in data]

def create_df(date,stock_no):
    s = pd.DataFrame(get_stock_history(date, stock_no))
    s.columns = ['date', 'shares', 'amount', 'open', 'high', 'low', 'close', 'change', 'turnover']
                #"日期","成交股數","成交金額","開盤價","最高價","最低價","收盤價","漲跌價差","成交筆數" 
    stock = []
    for i in range(len(s)):
        stock.append(stock_no)
    s['stockno'] = pd.Series(stock ,index=s.index)  #新增股票代碼欄，之後所有股票進入資料表才能知道是哪一張股票
    datelist = []
    for i in range(len(s)):
        datelist.append(s['date'][i])
    s.index = datelist  #索引值改成日期
    s2 = s.drop(['date'],axis = 1)  #刪除日期欄位
    mlist = []
    for item in s2.index:
        mlist.append(item.month)
    s2['month'] = mlist  #新增月份欄位
    return s2
        
listDji = ['2330']
for i in range(len(listDji)):
    result = create_df('20230430', listDji[i])
    print(result)
    
print(result.groupby('month').close.count())  #每個月幾個營業日
print(result.groupby('month').shares.sum())  #每個月累計成交股數

              shares       amount   open   high    low  close  change  \
2023-04-06  26706549  14130223516  530.0  531.0  526.0  530.0    -3.0   
2023-04-07  11114272   5903807797  535.0  535.0  529.0  531.0     1.0   
2023-04-10   8203168   4350907176  533.0  533.0  528.0  529.0    -2.0   
2023-04-11  23087326  12101146147  522.0  527.0  522.0  524.0    -5.0   
2023-04-12  24772925  12870788209  523.0  524.0  517.0  520.0    -4.0   
2023-04-13  27501436  14121978133  515.0  517.0  510.0  510.0   -10.0   
2023-04-14  21174653  10936462759  516.0  520.0  513.0  516.0     6.0   
2023-04-17  17895837   9264789087  519.0  520.0  514.0  520.0     4.0   
2023-04-18  15687059   8097905529  518.0  520.0  514.0  515.0    -5.0   
2023-04-19  23209923  11881404238  517.0  518.0  509.0  510.0    -5.0   
2023-04-20  21002685  10780268259  512.0  516.0  510.0  513.0     3.0   
2023-04-21  25440179  13103618008  520.0  520.0  510.0  511.0    -2.0   
2023-04-24  17579661   8903570323  505.0  510.0  50